# Perspectives on COVID-19: State Data

I am curious about COVID-19, and this notebook is my effort to find context and perspective from responsible, public data.

All data used is from:

- [The *New York Times*](https://github.com/nytimes/covid-19-data)
- [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm)
- [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)
- [South Carolina Department of Health and Environmental Control (DHEC)](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)
- [The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales)

## Configuring Libraries for the Almond Kernel

First, we'll make a bintray repository with libraries available to your almond kernel.

In [ ]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")

interp.repositories() ++= Seq(myBT)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [ ]:
import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source

import java.text.SimpleDateFormat
import java.util.Date


## Useful Functions

Save a string to a names file:

In [ ]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

Like `.split`, but preserving the character we split on:

In [ ]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

Pretty Print Things:

In [ ]:
def showMe(v:Any):Unit = {
  v match {
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

## Load Some Data

Load up-to-date data from the NY Times. Source: <https://github.com/nytimes/covid-19-data>.

In [ ]:
val dataLines: Vector[String] = {
    scala.io.Source.fromURL("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv").mkString.split("\n").toVector
    //scala.io.Source.fromURL("https://raw.githubusercontent.com/nytimes/covid-19-data/master/live/us-states.csv").mkString.split("\n").toVector
}

// quick test
val badLines = dataLines.filter( l => {
    l.split(",").size != 5
})

assert ( badLines.size == 0 )

## Comparative Data

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)

### USA

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

In [ ]:
val usa_2017_Total_Deaths: Int = 2813502

// Chronic lower respiratory diseases
val usa_2017_Total_Deaths_CLRD: Int = 160201

// Influenza and Pneumonia
val usa_2017_Total_Deaths_IPn: Int = 55672

// All respiratory-related deaths
val usa_2017_Total_Deaths_Resp: Int = usa_2017_Total_Deaths_CLRD + usa_2017_Total_Deaths_IPn

// Suicide
val usa_2017_Suicide_Total: Int = 47173

// Heart Disease
val usa_2017_Heart_Total: Int = 647457

// Accidents
val usa_2017_Accident_Total: Int = 169936

// Cancer
val usa_2017_Cancer_Total: Int = 599108



## Make Data Structures

For State data-points from the NY Times:

In [ ]:
case class StateDatum( date: String, state: String, cases: Int, deaths: Int)

For Aggregate data (just a Vector of the above):

In [ ]:
case class RunningTally( days: Vector[StateDatum] )

A daily snapshot (for the USA or a single state)

In [ ]:
case class DailySnapshot( date: String, newCases: Int, newDeaths: Int, totalCases: Int, totalDeaths: Int)

## Load Data

In [ ]:
val vst: Vector[StateDatum] = dataLines.tail.map( dl => {
    val fields: Vector[String] = dl.split(",").toVector
    val date: String = fields(0)
    val state: String = fields(1)
    val cases: Int = fields(3).toInt
    val deaths: Int = fields(4).toInt
    StateDatum(date, state, cases, deaths)
})

val rt = RunningTally(vst)

val stateTotalDeaths: Map[String, Int] = {
    val fileName = "state_deaths_2017.csv"
    scala.io.Source.fromFile(fileName).mkString.split("\n").toVector.tail.map( l => {
        val state: String = l.split(",").toVector(0)
        val deaths: Int = l.split(",").toVector(3).toInt
        (state, deaths)
    }).toMap
}

## Population Data 2018

Source: [United States Census Bureau](https://www.census.gov/newsroom/press-kits/2018/pop-estimates-national-state.html)

In [ ]:
val statePopulationMap: Map[String, Int] = Map("Alabama" -> 4887871, "Alaska" -> 737438, "Arizona" -> 7171646, "Arkansas" -> 3013825, "California" -> 39557045, "Colorado" -> 5695564, "Connecticut" -> 3572665, "Delaware" -> 967171, "District of Columbia" -> 702455, "Florida" -> 21299325, "Georgia" -> 10519475, "Hawaii" -> 1420491, "Idaho" -> 1754208, "Illinois" -> 12741080, "Indiana" -> 6691878, "Iowa" -> 3156145, "Kansas" -> 2911505, "Kentucky" -> 4468402, "Louisiana" -> 4659978, "Maine" -> 1338404, "Maryland" -> 6042718, "Massachusetts" -> 6902149, "Michigan" -> 9995915, "Minnesota" -> 5611179, "Mississippi" -> 2986530, "Missouri" -> 6126452, "Montana" -> 1062305, "Nebraska" -> 1929268, "Nevada" -> 3034392, "New Hampshire" -> 1356458, "New Jersey" -> 8908520, "New Mexico" -> 2095428, "New York" -> 19542209, "North Carolina" -> 10383620, "North Dakota" -> 760077, "Ohio" -> 11689442, "Oklahoma" -> 3943079, "Oregon" -> 4190713, "Pennsylvania" -> 12807060, "Rhode Island" -> 1057315, "South Carolina" -> 5084127, "South Dakota" -> 882235, "Tennessee" -> 6770010, "Texas" -> 28701845, "Utah" -> 3161105, "Vermont" -> 626299, "Virginia" -> 8517685, "Washington" -> 7535591, "West Virginia" -> 1805832, "Wisconsin" -> 5813568, "Wyoming" -> 577737)

val usaPopulation: Int = 327167434

## Data Functions

Taking, by default, our running-tally data (`rt`) and an optional `Option[String]` that can specify a state (defaults to `None`), return a `Vector[DailySnapshot]`, which will include new cases, new deaths, total cases, and total deaths.

In [ ]:
def totalNewDaily(data: Vector[StateDatum] = rt.days, state: Option[String] = None): Vector[DailySnapshot] = {
    val sortedData: Vector[StateDatum] = {
        val filtered: Vector[StateDatum] = {
            state match {
                case Some(s) => data.filter(_.state == s)
                case None => {
                    val groupedByDay: Vector[(String, Vector[StateDatum])] = {
                        data.groupBy(_.date).toVector
                    }
                    val merged: Vector[StateDatum] = groupedByDay.map( gbd => {
                        val vec = gbd._2
                        val vecDate: String = vec.head.date
                        val vecState: String = "USA"
                        val vecCases: Int = vec.map(_.cases).sum
                        val vecDeaths: Int = vec.map(_.deaths).sum
                        StateDatum(vecDate, vecState, vecCases, vecDeaths)
                    })
                    merged
                }
            }
        }
        val sorted: Vector[StateDatum] = filtered.sortBy(_.date)
        sorted
    }
    // We don't want a running, cumulative tally, but new cases/deaths each day
    sortedData.zipWithIndex.map( sd => {
        val d: StateDatum = sd._1
        val i: Int = sd._2
        val newCases: Int = {
            if (i == 0) d.cases
            else {
                val totalToday: Int = d.cases
                val totalPrev: Int = {
                    sortedData(i-1).cases
                }
                totalToday - totalPrev
            }
        }
        val newDeaths: Int = {
            if (i == 0) d.deaths
            else {
                val totalToday: Int = d.deaths
                val totalPrev: Int = {
                    sortedData(i-1).deaths
                }
                totalToday - totalPrev
            }
        }
        DailySnapshot(d.date, newCases, newDeaths, d.cases, d.deaths)
    })
    
}

## Any State: Cumulative Cases & Deaths

Edit the value for `state` below:

In [ ]:
val state: String = "Florida"

val threshold: Int = 14

In [ ]:
val oneStateAllCases: Vector[Int] = totalNewDaily(state = Some(state)).map(_.totalCases)
val oneStateAllDeaths: Vector[Int] = totalNewDaily(state = Some(state)).map(_.totalDeaths)



val oneStateAllCases_trace = Scatter(
  (1 to oneStateAllCases.size),
  oneStateAllCases,
  name = s"${state} Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0,0,204,1.0)
  )
)




val oneSate_active_cases: Vector[Int] = {
    val ac: Vector[Int] = oneStateAllCases.zipWithIndex.map( z => {
        val cases = z._1
        val index = z._2
        if (index <= threshold){
            cases
        } else {
            oneStateAllCases(index) - oneStateAllCases(index - threshold)
        }
    })
    ac
}

val oneStateActiveCases_trace = Scatter(
  (1 to oneSate_active_cases.size),
  oneSate_active_cases,
  name = s"${state} Active Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0,204,0,1.0)
  )
)




val oneStateAllDeaths_trace = Scatter(
  (1 to oneStateAllDeaths.size),
  oneStateAllDeaths,
  name = s"${state} Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204,0,0,1.0)
  )
)



val oneStateData = Vector(oneStateAllCases_trace, oneStateAllDeaths_trace, oneStateActiveCases_trace)

val oneStateLayout = Layout(s"${state}: COVID-19 Cases & Deaths (threshold = ${threshold} days)")

plot(oneStateData, oneStateLayout)



## Compare Any Three States: Total Deaths

Edit the value for `states`, immediately below, to specify three states to compare.

In [ ]:
val states:Vector[String] = Vector(
    "Florida",
    "Georgia",
    "Massachusetts"
)

val adjustForPopulation: Boolean = false

case class StateNumbers(index: Int, state:String, nums:Vector[Int])

// We need to start all the states on the same day…
def padVector(shortVector: Vector[Int], maxSize: Int): Vector[Int] = {
    Vector(1)
}

// Get a data for each state
val state_death_vec: Vector[StateNumbers] = states.zipWithIndex.map( z => {
    val s: String = z._1
    val i: Int = z._2
    StateNumbers(i, s, totalNewDaily(state = Some(s)).map(_.totalDeaths))
})

// Normalize the size
val state_death_vec_normalized: Vector[StateNumbers] = {
    val maxSize = state_death_vec.sortBy(_.nums.size).last.nums.size
    state_death_vec.map( sdv => {
        val s = sdv.state
        val n = sdv.nums
        val i = sdv.index
        val howMany = maxSize - n.size
        val adder = Vector.fill(howMany)(0)
        val newN = adder ++ n
        StateNumbers(i, s, newN)
    }).sortBy(_.nums.last)
}

// Normalize for Population
val state_death_vec_pop_adj: Vector[StateNumbers] = {
    val minPopState: String = {
        states.map( s => {
            (s, statePopulationMap(s))
        }).sortBy(_._2).head._1
    }
    println(s"Min population: ${minPopState}")
    val minPop: Int = statePopulationMap(minPopState)
    state_death_vec_normalized.map( v => {
        val i = v.index
        val s = v.state
        val p = statePopulationMap(s)
        val m = p / minPop
        val nums: Vector[Int] = v.nums.map( n => {
            n / m
        }) 
        StateNumbers(i, s, nums)
    }).sortBy(_.nums.last)
}

val colors: Vector[(Int, Int, Int, Double)] = {
    Vector(
        (204,0,0,0.95),
        (0,0,204,0.95),
        (0,204,0,0.95),
    )
}

val colors2: Vector[(Int, Int, Int, Double)] = {
    Vector(
        (204,0,0,0.55),
        (0,0,204,0.55),
        (0,204,0,0.55),
    )
}

val statePlotters: Vector[Scatter] = state_death_vec_normalized.map( sd => {
    Scatter(
  (1 to sd.nums.size),
  sd.nums,
  name = s"${sd.state}",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(
        colors(sd.index)._1,
        colors(sd.index)._2,
        colors(sd.index)._3,
        colors(sd.index)._4
    ),
  )
)
})

val statePlotters2: Vector[Scatter] = state_death_vec_pop_adj.map( sd => {
    Scatter(
  (1 to sd.nums.size),
  sd.nums,
  name = s"${sd.state} Adj.",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(
        colors(sd.index)._1,
        colors(sd.index)._2,
        colors(sd.index)._3,
        colors(sd.index)._4
    ),
  )
)
})


val data = {
    if (adjustForPopulation) statePlotters2
    else statePlotters
}

val layout = Layout("COVID-19 Deaths: " + states.mkString(", "))

plot(data, layout)

## South Carolina Data

Source for 2018 South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm) and [SC DHEC](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)

In [ ]:
// Total Deaths
val sc_2018_Total_Deaths: Int = 50633

// Chronic Lower Respiratory Disease
val sc_2017_Total_Deaths_CLRD: Int = 2990

// Influenza and Pneumonia
val sc_2017_Total_Deaths_IPn: Int = 882

// All Respiratory-Related Deaths
val sc_2017_Total_Deaths_All_Resp = sc_2017_Total_Deaths_CLRD + sc_2017_Total_Deaths_IPn

// SC Covid Death Toll
val sc_covid_death_toll = totalNewDaily(state = Some("South Carolina")).map(_.totalDeaths)

// SC Total Cases
val sc_covid_total_cases = totalNewDaily(state = Some("South Carolina")).map(_.totalCases)




## Compare Any Three States: Daily Deaths

Edit the value for `states`, immediately below, to specify three states to compare.

In [ ]:
val statesNew:Vector[String] = Vector(
    "South Carolina",
    "Georgia",
    "Massachusetts"
)

// Get a data for each state
val state_new_death_vec: Vector[StateNumbers] = statesNew.zipWithIndex.map( z => {
    val s: String = z._1
    val i: Int = z._2
    StateNumbers(i, s, totalNewDaily(state = Some(s)).map(_.newDeaths))
})

// Normalize the size
val state_new_death_vec_normalized: Vector[StateNumbers] = {
    val maxSize = state_new_death_vec.sortBy(_.nums.size).last.nums.size
    state_new_death_vec.map( sdv => {
        val s = sdv.state
        val n = sdv.nums
        val i = sdv.index
        val howMany = maxSize - n.size
        val adder = Vector.fill(howMany)(0)
        val newN = adder ++ n
        StateNumbers(i, s, newN)
    })
}

val colors: Vector[(Int, Int, Int, Double)] = {
    Vector(
        (204,0,0,0.95),
        (0,0,204,0.95),
        (0,204,0,0.95),
    )
}

val statePlottersNew: Vector[Scatter] = state_new_death_vec_normalized.map( sd => {
    Scatter(
  (1 to sd.nums.size),
  sd.nums,
  name = s"${sd.state}",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(
        colors(sd.index)._1,
        colors(sd.index)._2,
        colors(sd.index)._3,
        colors(sd.index)._4
    ),
  )
)
})

val dataNew = statePlottersNew

val layoutNew = Layout("COVID-19 Daily Deaths: " + statesNew.mkString(", "))

plot(dataNew, layoutNew)

## Compare Any Three States: Daily Deaths (N-day avg.)

Edit the value for `states`, immediately below, to specify three states to compare.

In [ ]:
val nDays: Int = 14

val statesNew:Vector[String] = Vector(
    "Florida",
    "Georgia",
    "Massachusetts"
)

// Get a data for each state
val state_new_death_vec: Vector[StateNumbers] = statesNew.zipWithIndex.map( z => {
    val s: String = z._1
    val i: Int = z._2
    StateNumbers(i, s, totalNewDaily(state = Some(s)).map(_.newDeaths))
})

// Normalize the size
val state_new_death_vec_normalized: Vector[StateNumbers] = {
    val maxSize = state_new_death_vec.sortBy(_.nums.size).last.nums.size
    state_new_death_vec.map( sdv => {
        val s = sdv.state
        val n = sdv.nums
        val i = sdv.index
        val howMany = maxSize - n.size
        val adder = Vector.fill(howMany)(0)
        val newN = adder ++ n
        StateNumbers(i, s, newN)
    })
}

val colors: Vector[(Int, Int, Int, Double)] = {
    Vector(
        (204,0,0,0.95),
        (0,0,204,0.95),
        (0,204,0,0.95),
    )
}

val statePlottersNew: Vector[Scatter] = state_new_death_vec_normalized.map( sd => {
    Scatter(
  (1 to sd.nums.size),
  sd.nums.sliding(nDays,1).toVector.map( n => {
      n.sum / n.size
  }),
  name = s"${sd.state}",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(
        colors(sd.index)._1,
        colors(sd.index)._2,
        colors(sd.index)._3,
        colors(sd.index)._4
    ),
  )
)
})

val dataNew = statePlottersNew

val layoutNew = Layout(s"Daily Deaths (${nDays}-day running average): " + statesNew.mkString(", "))

plot(dataNew, layoutNew)

## SC: COVID vs. Flu-Season

Throughout a year like 2017, the death-toll for Influenzas and Pneumonias mounts, day to day. Here we chart that death-toll and compare it to the reported COVID-19 toll, assuming the annual deaths are evenly distributed across the year. 

But the flu-season is, as defined by the CDC, 6 months, from October through March. So we can also compare daily flu deaths and their mounting toll, assuming a six-month flu-season.

Source for 2018 South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm) and [SC DHEC](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)

In [ ]:


val sc_2018_mounting_flu_toll: Vector[Int] = {
    val daily = sc_2017_Total_Deaths_IPn / 365
    (1 to sc_covid_death_toll.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val sc_2018_mounting_flu_toll_6mo: Vector[Int] = {
    val daily = sc_2017_Total_Deaths_IPn / 365 * 2
    (1 to sc_covid_death_toll.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val sc_2018_mounting_flu_toll_trace = Scatter(
  (1 to sc_covid_death_toll.size),
  sc_2018_mounting_flu_toll,
  name = "2018 Influenza Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val sc_2018_mounting_flu_toll_6mo_trace = Scatter(
  (1 to sc_covid_death_toll.size),
  sc_2018_mounting_flu_toll_6mo,
  name = "2018 Influenza Deaths (6 mo. season)",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val sc_2020_covid_toll_trace = Scatter(
  (1 to sc_covid_death_toll.size),
  sc_covid_death_toll,
  name = "2020 Covid Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 0, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(
        sc_2018_mounting_flu_toll_trace,
        sc_2018_mounting_flu_toll_6mo_trace,
        sc_2020_covid_toll_trace
)

val layout = Layout("South Carolina: COVID Death Toll vs. Flu Season, 2018")

plot(data, layout)

### New York

Source for 2018 New York Data: [“Stats of the State of New York”, CDC](https://www.cdc.gov/nchs/pressroom/states/newyork/newyork.htm), and [“Vital Statistics”, New York State, Department of Health](https://www.health.ny.gov/statistics/vital_statistics/2017/table35.htm)

In [ ]:
// Total Deaths
val ny_2017_Total_Deaths: Int = 155191

// Chronic Lower Respiratory Disease
val ny_2017_Total_Deaths_CLRD: Int = 7258

// Influenza and Pneumonia
val ny_2017_Total_Deaths_IPn: Int = 4517

// All Respiratory-Related Deaths
val ny_2017_Total_Deaths_All_Resp = ny_2017_Total_Deaths_CLRD + ny_2017_Total_Deaths_IPn

// NY Covid Death Toll
val ny_covid_death_toll = totalNewDaily(state = Some("New York")).map(_.totalDeaths)

// NY Total Cases
val ny_covid_total_cases = totalNewDaily(state = Some("New York")).map(_.totalCases)

## New York: COVID vs. Flu-Season

Source for 2018 New York Data: [“Stats of the State of New York”, CDC](https://www.cdc.gov/nchs/pressroom/states/newyork/newyork.htm), and [“Vital Statistics”, New York State, Department of Health](https://www.health.ny.gov/statistics/vital_statistics/2017/table35.htm)

Throughout a year like 2017, the death-toll for Influenzas and Pneumonias mounts, day to day. Here we chart that death-toll and compare it to the reported COVID-19 toll, assuming the annual deaths are evenly distributed across the year. 

But the flu-season is, as defined by the CDC, 6 months, from October through March. So we can also compare daily flu deaths and their mounting toll, assuming a six-month flu-season.

Source for 2018 South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm) and [SC DHEC](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)

In [ ]:

val ny_2018_allDeaths = 155358

val ny_2018_all_toll: Vector[Int] = {
    val daily = stateTotalDeaths("New York") / 365
    (1 to ny_covid_death_toll.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    }) 
}

val ny_2018_mounting_flu_toll: Vector[Int] = {
    val daily = ny_2017_Total_Deaths_IPn / 365
    (1 to ny_covid_death_toll.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val ny_2018_mounting_flu_toll_6mo: Vector[Int] = {
    val daily = ny_2017_Total_Deaths_IPn / 365 * 2
    (1 to ny_covid_death_toll.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val ny_2018_mounting_flu_toll_trace = Scatter(
  (1 to ny_covid_death_toll.size),
  ny_2018_mounting_flu_toll,
  name = "2017 Influenza Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val ny_2018_mounting_Alltoll_trace = Scatter(
  (1 to ny_2018_all_toll.size),
  ny_2018_all_toll,
  name = "2018 All Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val ny_2018_mounting_flu_toll_6mo_trace = Scatter(
  (1 to ny_covid_death_toll.size),
  ny_2018_mounting_flu_toll_6mo,
  name = "2017 Influenza Deaths (6 mo. season)",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val ny_2020_covid_toll_trace = Scatter(
  (1 to ny_covid_death_toll.size),
  ny_covid_death_toll,
  name = "2020 Covid Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 0, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(
        ny_2018_mounting_flu_toll_trace,
        ny_2018_mounting_flu_toll_6mo_trace,
        ny_2020_covid_toll_trace,
        ny_2018_mounting_Alltoll_trace
)

val layout = Layout("New York: COVID Death Toll vs. Flu Season, 2018")

plot(data, layout)

## One State Case-Fatality-Rate

A rolling calculation of deaths/cases. Edit the state name immediately below:

In [ ]:
val oneStateCfr: String = "South Carolina"

In [ ]:
val one_state_total_cases: Vector[Int] = totalNewDaily(state = Some(oneStateCfr)).map(_.totalCases)
val one_state_total_deaths: Vector[Int] = totalNewDaily(state = Some(oneStateCfr)).map(_.totalDeaths)

val one_state_cfr: Vector[Double] = one_state_total_deaths.zip(one_state_total_cases).map( c => {
    c._1.toDouble / c._2.toDouble
})

val case_fatality_rate = Scatter(
  (1 to one_state_cfr.size),
  one_state_cfr,
  name = "Case Fatality Rate",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(case_fatality_rate)

val layout = Layout(s"2020 Covid CFR: ${oneStateCfr}")

plot(data, layout)

## Comparative CFR

Since testing is still constrained, these numbers should be expected to suffer from “ascertainment bias”, since only those presenting symptoms, and often only those presenting severe symptoms, get tested.

In [ ]:
val usa_total_cases: Vector[Int] = totalNewDaily(state = None).map(_.totalCases)
val usa_total_deaths: Vector[Int] = totalNewDaily(state = None).map(_.totalDeaths)

val cfr_usa: Vector[Double] = usa_total_deaths.zip(usa_total_cases).map( c => {
    c._1.toDouble / c._2.toDouble
})


val usa_case_fatality_rate = Scatter(
  (1 to cfr_usa.size),
  cfr_usa,
  name = "USA",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

// -------- States below ----------

val statesForCfr:Vector[String] = Vector(
    "Michigan",
    "South Carolina",
    "Florida"
)

case class StateCfr(index: Int, state: String, cfrs: Vector[Double])

// Get a data for each state
val state_cfr_vec: Vector[StateCfr] = statesForCfr.zipWithIndex.map( z => {
    val i = z._2
    val s = z._1
    val cfrs: Vector[Double] = totalNewDaily(state = Some(s)).map( d => {
        val deaths: Int = d.totalDeaths
        val cases: Int = d.totalCases
        deaths.toDouble / cases.toDouble
    })
    StateCfr(i, s, cfrs)
})

// Normalize the size
val state_new_death_vec_normalized: Vector[StateCfr] = {
    val maxSize = cfr_usa.size
    state_cfr_vec.map( sdv => {
        val s = sdv.state
        val n = sdv.cfrs
        val i = sdv.index
        val howMany = maxSize - n.size
        val adder = Vector.fill(howMany)(0.0)
        val newN = adder ++ n
        StateCfr(i, s, newN)
    })
}

val colors: Vector[(Int, Int, Int, Double)] = {
    Vector(
        (204,0,0,0.95),
        (0,0,204,0.95),
        (0,204,0,0.95),
    )
}


val statePlottersNew: Vector[Scatter] = state_new_death_vec_normalized.map( sd => {
    Scatter(
  (1 to sd.cfrs.size),
  sd.cfrs,
  name = s"${sd.state}",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(
        colors(sd.index)._1,
        colors(sd.index)._2,
        colors(sd.index)._3,
        colors(sd.index)._4
    ),
  )
)
})

val dataNew = statePlottersNew :+ usa_case_fatality_rate

val layoutNew = Layout("COVID-19 Case Fatility Rate: " + statesForCfr.mkString(", "))

plot(dataNew, layoutNew)
